# quick stocktake of UK LLC occupation data

In [1]:
# packages
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import exc
from sqlalchemy.engine import URL
from IPython.display import display, Markdown

In [2]:
# DB connection info
def connect():
    '''
    Connect to the database with sql alchemy, system ODBC connection.
    Returns:
        engine (sqlalchemy object): engine for database operations
    '''
    connection_string = (
    r"DSN=LLC_DB"
    )
    connection_url = URL.create(
        "mssql+pyodbc",
        query={"odbc_connect": connection_string}
    )
    engine = create_engine(connection_url, isolation_level="AUTOCOMMIT")
    return engine

In [3]:
# connect to DB
cnxn = connect()
# query to pull all labs
sel_q = 'select * from DESCS.all_descriptions'
# read all labels
all_labs = pd.read_sql(sel_q, cnxn)

In [4]:
# occupation keywords define
occ_key = ['soc', 'soc2', 'sic', 'sic2', 'occupation', 'job']
# process list into format convenient fo search
occ_pat = ['\\b' + i + '\\b' for i in occ_key]
occ_pat = '|'.join(occ_pat)

In [5]:
# search
all_labs['occ_flag'] = all_labs['variable_label'].str.contains(occ_pat, case = False)
# filter the matches
occ_labs = all_labs.loc[all_labs['occ_flag'] == True]

In [6]:
# remove duplicate versions
occ_labs['table_name1'] = occ_labs['table_name'].str.split('_v000').str[0]
# drop unreq vars
occ_labs = occ_labs.drop(columns = ['avail_from_dt', 'table_name', 'occ_flag'], axis = 1)
# rename 
occ_labs = occ_labs.rename(columns = {'table_name1' : 'table_name',
                                     'TABLE_SCHEMA' : 'data_source'})
# remove dups
occ_labs = occ_labs.drop_duplicates()
# sort
occ_labs = occ_labs.sort_values(by = ['data_source', 'table_name', 'variable_name'])


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
# group by study
study_group = occ_labs.groupby(by = 'data_source').size().reset_index(name = 'variable_count')
# display the dataframe
print(study_group.to_markdown(index=False, tablefmt="fancy_grid"))

╒════════════════╤══════════════════╕
│ data_source    │   variable_count │
╞════════════════╪══════════════════╡
│ ALSPAC         │               16 │
├────────────────┼──────────────────┤
│ BCS70          │              422 │
├────────────────┼──────────────────┤
│ BIB            │               26 │
├────────────────┼──────────────────┤
│ ELSA           │              145 │
├────────────────┼──────────────────┤
│ EPICN          │                9 │
├────────────────┼──────────────────┤
│ EXCEED         │                9 │
├────────────────┼──────────────────┤
│ FENLAND        │               12 │
├────────────────┼──────────────────┤
│ GENSCOT        │               16 │
├────────────────┼──────────────────┤
│ GLAD           │                1 │
├────────────────┼──────────────────┤
│ MCS            │              706 │
├────────────────┼──────────────────┤
│ NCDS58         │             1416 │
├────────────────┼──────────────────┤
│ NEXTSTEP       │              474 │
├───────────

**Table 1** Number of variables per **data source** with match on occupation keywords

In [10]:
# group by study + table_name
table_group = occ_labs.groupby(by = ['data_source', 'table_name']).size().reset_index(name = 'variable_count')
# display the dataframe
print(table_group.to_markdown(index=False, tablefmt="fancy_grid"))

╒════════════════╤══════════════════════════════╤══════════════════╕
│ data_source    │ table_name                   │   variable_count │
╞════════════════╪══════════════════════════════╪══════════════════╡
│ ALSPAC         │ custom_occupationm           │                1 │
├────────────────┼──────────────────────────────┼──────────────────┤
│ ALSPAC         │ custom_occupationy           │                1 │
├────────────────┼──────────────────────────────┼──────────────────┤
│ ALSPAC         │ wave0m                       │                1 │
├────────────────┼──────────────────────────────┼──────────────────┤
│ ALSPAC         │ wave0y                       │                1 │
├────────────────┼──────────────────────────────┼──────────────────┤
│ ALSPAC         │ wave1m                       │                1 │
├────────────────┼──────────────────────────────┼──────────────────┤
│ ALSPAC         │ wave1y                       │                1 │
├────────────────┼────────────────

**Table 2** Number of variables per **dataset** with match on occupation keywords

In [9]:
# save out the full dataset
# occ_labs.to_csv('S:/UKLLC - UKLLC Databank/misc/occupation_data/stocktake.csv', index = False)